# **Projeto Airbnb Rio** - Ferramenta de Previsão de Preço de Imóvel para pessoas comuns 

**Meu objetivo**
Construir um modelo de previsão de preço que permita uma pessoa comum que possui um imóvel possa saber quanto deve cobrar pela diária do seu imóvel.

Ou ainda, para o locador comum, dado o imóvel que ele está buscando, ajudar a saber se aquele imóvel está com preço atrativo (abaixo da média para imóveis com as mesmas características) ou não.

**O que temos disponível.**
As bases de dados foram retiradas do site kaggle: https://www.kaggle.com/allanbruno/airbnb-rio-de-janeiro

Elas estão disponíveis para download

* As bases de dados são os preços dos imóveis obtidos e suas respectivas características em cada mês.
* Os preços são dados em reais (R$)
* Temos bases de abril de 2018 a maio de 2020, com exceção de junho de 2018 que não possui base de dados

**Expectativas Iniciais**
* Acredito que a sazonalidade pode ser um fator importante, visto que meses como dezembro costumam ser bem caros no RJ
* A localização do imóvel deve fazer muita diferença no preço, já que no Rio de Janeiro a localização pode mudar completamente as características do lugar (segurança, beleza natural, pontos turísticos)
* Adicionais/Comodidades podem ter um impacto significativo, visto que temos muitos prédios e casas antigos no Rio de Janeiro

Vamos descobrir o quanto esses fatores impactam e se temos outros fatores não tão intuitivos que são extremamente importantes.

In [1]:
#Bibliotecas importadas...
import pandas as pd #fazer análises e editar o dataframe
import pathlib #auxilia no caminho de base

In [ ]:
meses = {'jan':1,'fev':2,'mar':3,'abr':4,'mai':5,'jun':6,'jul':7,'ago':8,
         'set':9,'out':10,'nov':11,'dez':12}

caminho_bases = pathlib.Path(r'/kaggle/input/airbnb-rio-de-janeiro')
base_airbnb = pd.DataFrame()


for arquivo in caminho_bases.iterdir():
    
    if arquivo.name != "total_data.csv":
        nome_mes = arquivo.name[:3]
        mes = meses[nome_mes]
        
        ano = arquivo.name[-8:]
        ano = int(ano.replace('.csv',''))
        
        df = pd.read_csv(caminho_bases / arquivo.name)
        df['ano'] = ano
        df['mes'] = mes
        base_airbnb = base_airbnb.append(df)
    else:
        pass
display(base_airbnb)

* Como tem muitas colunas, o modelo pode acabar ficando lento.
* 
* Além disso, uma análise rápida permite ver que várias colunas não são necessárias para o modelo de previão, por isso, vou excluir algumas colunas da base
* 
* Tipos de colunas:

1. IDs, links e informações não relevantes para o modelo
1. Colunas repetidas ou extremamente parecidas com outras (que dão a mesmas informações)
1. colunas preenchidas com texto livre -> Não irei rodar nenhuma na análise de palavras ou algo do tipo
1. Colunas com 50% ou mais (none)

* Para isso vou criar um arquivo excel com as 1000 primeiros registros e fazer uma análise qualitativa

In [ ]:
#Permite análisar os primeiros 1000 registro dentro do excel e retirar as coisas que não fazem sentido
#para o modelo
base_airbnb.head(1000).to_csv('primeiros registros',sep=';')
primeiros_registro = pd.read_csv(r'/kaggle/working/primeiros registros',sep=';')
display(primeiros_registro)

**DEPOIS DA ANÁLISE QUALITATIVA DAS COLUNAS, LEVANDO EM CONTA OS CRITÉRIOS EXPLICADOS ACIMA, FIQUEI COM AS SEGUINTES COLUNAS:**

In [4]:
colunas = ['host_response_time','host_response_rate','host_is_superhost',
           'host_listings_count','latitude','longitude','property_type',
           'room_type','accommodates','bathrooms','bedrooms','beds','bed_type',
           'amenities','price','security_deposit','cleaning_fee','guests_included',
           'extra_people','minimum_nights','maximum_nights','number_of_reviews',
           'review_scores_rating','review_scores_accuracy','review_scores_cleanliness',
           'review_scores_checkin','review_scores_communication','review_scores_location',
           'review_scores_value','instant_bookable','is_business_travel_ready','cancellation_policy',
           'ano','mes']

base_airbnb = base_airbnb.loc[:,colunas]
display(base_airbnb)

,host_response_time,host_response_rate,host_is_superhost,host_listings_count,latitude,longitude,property_type,room_type,accommodates,bathrooms,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,ano,mes
0,within an hour,100%,f,2.0,-22.965919,-43.178962,Condominium,Entire home/apt,5,1.0,...,9.0,10.0,10.0,10.0,9.0,t,f,strict_14_with_grace_period,2019,2
1,within a day,100%,t,3.0,-22.977117,-43.190454,Apartment,Entire home/apt,3,1.0,...,10.0,9.0,10.0,10.0,9.0,f,f,strict_14_with_grace_period,2019,2
2,within an hour,100%,t,1.0,-22.983024,-43.214270,Apartment,Entire home/apt,3,1.0,...,10.0,10.0,10.0,10.0,10.0,t,f,strict_14_with_grace_period,2019,2
3,within an hour,100%,f,8.0,-22.998669,-43.271372,House,Entire home/apt,15,4.0,...,NaN,NaN,NaN,NaN,NaN,f,f,strict_14_with_grace_period,2019,2
4,within a few hours,100%,t,1.0,-22.988165,-43.193588,Apartment,Entire home/apt,3,1.5,...,10.0,10.0,10.0,10.0,9.0,f,f,strict_14_with_grace_period,2019,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38169,NaN,NaN,f,3.0,-22.922762,-43.178087,Apartment,Private room,2,1.0,...,NaN,NaN,NaN,NaN,NaN,f,f,moderate,2018,8
38170,NaN,NaN,f,1.0,-22.820039,-43.375005,Loft,Entire home/apt,4,3.0,...,NaN,NaN,NaN,NaN,NaN,f,f,flexible,2018,8
38171,NaN,NaN,f,1.0,-22.945188,-43.182852,Apartment,Entire home/apt,2,1.0,...,NaN,NaN,NaN,NaN,NaN,f,f,flexible,2018,8
38172,NaN,NaN,f,2.0,-22.923240,-43.226054,Apartment,Entire home/apt,7,2.0,...,NaN,NaN,NaN,NaN,NaN,t,f,flexible,2018,8
